# Dados

## Caminhos

In [1]:
datasets_path     = '/homeLocal/praticas-cv-cnn/datasets/'
models_path       = '/homeLocal/praticas-cv-cnn/models/'
tensorboard_path  = '/homeLocal/praticas-cv-cnn/Tensorboard/lenet5/'

## Dataloader

In [2]:
from torch.utils.data import DataLoader
import torchvision

import matplotlib.pyplot as plt
import numpy as np

def my_imshow(img, dataset, numImages=10):
  
    if dataset == 'cifar10' : 
        img = img / 2 + 0.5     # unnormalize
    
    img = torchvision.utils.make_grid(img[:numImages],nrow=numImages//2)
    
    npimg = img.numpy()    
    npimg = np.transpose(npimg, (1, 2, 0))
    
    plt.axis('off')
    plt.imshow(npimg)
    plt.show()

def show_images(train_loader, test_loader, dataset, numImages=10) :
    print('Train samples')
    # get some random training images
    dataiter = iter(train_loader)
    images = next(dataiter)[0]
    my_imshow(images, dataset, numImages)

    print('Test samples')
    # get some random training images
    dataiter = iter(test_loader)
    images = next(dataiter)[0]
    my_imshow(images, dataset, numImages)

def get_data_cifar10 ( batch_size , show_image=False, numImages=10 ) :
  
    my_transform = torchvision.transforms.Compose([
                            torchvision.transforms.Resize(28),
                            torchvision.transforms.ToTensor(),
                            torchvision.transforms.Normalize(mean=[0.5],std=[0.5])
                                    ])

    train_dataset = torchvision.datasets.CIFAR10(
                                root=f'{datasets_path}/train/', 
                                train=True, 
                                transform=my_transform, 
                                download=False
                                )
    test_dataset = torchvision.datasets.CIFAR10(
                                root=f'{datasets_path}/test/',
                                train=False, 
                                transform=my_transform, 
                                download=False
                                )
    train_loader = DataLoader(train_dataset, 
                                batch_size=batch_size,
                                shuffle=True
                                )
    test_loader = DataLoader(test_dataset, 
                            batch_size=batch_size,
                            shuffle=False
                            )
    
    if show_image :
        show_images(train_loader, test_loader, 'cifar10', numImages)
    
    return train_loader, test_loader, len(train_dataset)


def get_data_mnist ( batch_size , show_image=False, numImages=10 ) :
  
    train_dataset = torchvision.datasets.mnist.MNIST(
                            root=f'{datasets_path}/train/', 
                            train=True, 
                            transform=torchvision.transforms.ToTensor(), 
                            download=False
                            )
    test_dataset = torchvision.datasets.mnist.MNIST(
                            root=f'{datasets_path}/test/',
                            train=False, 
                            transform=torchvision.transforms.ToTensor(), 
                            download=False
                            )

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    
    if show_image :
        show_images(train_loader, test_loader, 'mnist', numImages)
    
    return train_loader, test_loader, len(train_dataset)

In [3]:
get_data_mnist(batch_size=256, show_image=True, numImages=16);

RuntimeError: Dataset not found. You can use download=True to download it

In [ ]:
get_data_cifar10(batch_size=256, show_image=True, numImages=10);

# Rede

## Arquitetura

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class LeNet(nn.Module) :
  
    def __init__(self, num_classes=10, n_channels=1):
        super().__init__()
        
    def forward(self, x, debug=False):
        

## Informações sobre a rede

In [ ]:
import torch

if torch.cuda.is_available():
    my_device = torch.device("cuda:0")
else:
    my_device = torch.device("cpu")
    
print(f"Running on {my_device.type}.")

net = LeNet(num_classes=10, n_channels=1)
net = net.to(my_device)

a = torch.rand( (1, 3, 28, 28) )

b = net( a.to(my_device) )

In [ ]:
from torchsummary import summary
summary(net, input_size=(3,28,28), batch_size=256)
del net, a, b

## Treinamento

In [ ]:
from torch.utils.tensorboard import SummaryWriter

import torch.optim 

import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm

import copy
  
from datetime import datetime

def train ( dataset, epochs=100, lr=1e-1, prefix='', upper_bound=99.0, device='cpu',
           save=False, debug=False, plot_histograms=False, lambda_reg=0) :

    if dataset == 'mnist' :
        batch_size = 256
        train_loader, test_loader, dataset_size = get_data_mnist(batch_size, 
                                                                    show_image=True
                                                                    )
        num_classes = 10
        n_channels = 1
    elif dataset == 'cifar10' :
        batch_size = 256
        train_loader, test_loader, dataset_size = get_data_cifar10(batch_size, 
                                                                    show_image=True
                                                                    )
        num_classes = 10
        n_channels = 3
    else :
        print('Dataset loader not implemented.')
        return None    
  
    net = LeNet( num_classes, n_channels )
    net.to(device)

    optimizer = torch.optim.SGD(net.parameters(), lr, weight_decay=lambda_reg)
    loss = nn.CrossEntropyLoss()

    now = datetime.now()
    suffix = now.strftime("%Y%m%d_%H%M%S")
    prefix = prefix + '-' + suffix if prefix != '' else suffix

    writer = SummaryWriter( log_dir=tensorboard_path+prefix )
    
    writer.add_graph(net, next(iter(train_loader))[0].to(my_device))

    accuracies = []
    max_accuracy = -1.0

    for epoch in tqdm( range(epochs) , desc='Training epochs...' ) :
        net.train()  
        for idx, (train_x, train_label) in enumerate(train_loader):
            train_x = train_x.to(device)
            train_label = train_label.to(device)

            optimizer.zero_grad()

            predict_y = net( train_x )[0]

            # Loss:
            error = loss( predict_y , train_label.long() )

            writer.add_scalar( 'Loss/train', error, 
                            idx+( epoch*(dataset_size//batch_size) ) )

            error.backward()
            optimizer.step()

            # Accuracy:
            predict_ys = torch.max( predict_y, axis=1 )[1]
            correct    = torch.sum(predict_ys == train_label)

            writer.add_scalar( 'Accuracy/train', correct/train_x.size(0), 
                            idx+( epoch*(dataset_size//batch_size) ) )

            if debug and idx % 10 == 0 :
                print(f'idx: {idx}, _error: {error}')

        if plot_histograms : 
            plot_histograms_tensorboard(writer, net, epoch)
        
        accuracy = validate(net, test_loader, device=device)
        accuracies.append(accuracy.cpu())
        writer.add_scalar( 'Accuracy/test', accuracy, epoch )
    
        if accuracy > max_accuracy:
            best_model = copy.deepcopy(net)
            max_accuracy = accuracy
            print(f'Saving Best Model with Accuracy: {max_accuracy:3.4f}')
            
        print( f'Epoch: {epoch+1:3d} | Accuracy : {accuracy:3.4f}%' )

        if accuracy > upper_bound :
            break
   
    if save : 
        path = f'{models_path}LeNet5-{dataset}-{max_accuracy:.2f}.pkl'
        torch.save(best_model.state_dict(), path)
        print('Model saved in:',path)
  
    plt.plot(accuracies)

    writer.flush()
    writer.close()
    
    return best_model

In [ ]:
def plot_histograms_tensorboard ( writer, net, epoch ) :
    writer.add_histogram('Bias/conv1',   net.c1.bias,        epoch)
    writer.add_histogram('Weight/conv1', net.c1.weight,      epoch)
    writer.add_histogram('Grad/conv1',   net.c1.weight.grad, epoch)

    writer.add_histogram('Bias/conv3',   net.c3.bias,        epoch)
    writer.add_histogram('Weight/conv3', net.c3.weight,      epoch)
    writer.add_histogram('Grad/conv3',   net.c3.weight.grad, epoch)

    writer.add_histogram('Bias/conv5',   net.c5.bias,        epoch)
    writer.add_histogram('Weight/conv5', net.c5.weight,      epoch)
    writer.add_histogram('Grad/conv5',   net.c5.weight.grad, epoch)

## Validação

In [ ]:
def validate ( model , data , device='cpu') :
    model.eval()
    correct = 0
    sum = 0
    
    for idx, (test_x, test_label) in enumerate(data) : 
        test_x = test_x.to(device)
        test_label = test_label.to(device)
        predict_y = model( test_x )[0].detach()
        predict_ys = torch.max( predict_y, axis=1 )[1]
        sum = sum + test_x.size(0)
        correct = correct + torch.sum(predict_ys == test_label)
    
    return correct*100./sum

# Execução

## Treina

In [ ]:
if torch.cuda.is_available():
    my_device = torch.device("cuda:0")
else:
    my_device = torch.device("cpu")
print(f"Running on {my_device.type}.")

epochs = 10
dataset = 'mnist' # 'cifar10' 
lr = 1.3e0
lambda_reg = 0

prefix = 'LeNet-{}-e-{}-lr-{}'.format(dataset, epochs, lr)

net = train( dataset=dataset, epochs=epochs, lr=lr, prefix=prefix , upper_bound=100, device=my_device,
            save=True, debug=False, plot_histograms=True, lambda_reg=lambda_reg )

# Carregar Rede de arquivo

In [ ]:
del net

path = '/homeLocal/praticas-cv-cnn/models/LeNet5-mnist-95.52.pkl'
n_channels = 1

def load_LeNet ( device , path ) :
    net = LeNet(num_classes=10, n_channels=n_channels)
    net = net.to(device)
    net.load_state_dict(torch.load(path))
    net.eval()
    return net

net = load_LeNet(my_device, path)

# Carregar dado do MNIST e inferir

In [ ]:
import PIL
import torchvision
import numpy as np

def sample_and_predict ( seed=None ) :

    if seed is not None :
        np.random.seed(seed)

    dataset = torchvision.datasets.MNIST(
                                      root=f'{datasets_path}/test/', 
                                      train=False
                                      )
 
    i = np.random.randint(len(dataset))
    sample = dataset[i][0]

    x = torchvision.transforms.ToTensor()(sample).float()

    x = x.unsqueeze_(0)

    x = x.to(my_device)

    y = net ( x )[1]
    confidence = torch.max(y, 1)[0]
    prediction = torch.max(y, 1)[1]

    print( 'Sample: {}'.format(i) )
    plt.axis('off')
    plt.imshow( sample , cmap='gray')

    confidence = confidence.data.cpu().numpy()[0]
    prediction = prediction.data.cpu().numpy()[0]

    return prediction, confidence, dataset[i][1]

prediction, confidence, label = sample_and_predict()
print( f'\nPredicted clas: {prediction} \nClassifier confidence: {confidence*100:4.2f}% \nTrue label: {label}' )
